# Notebook to pull data from an Arkiver

Can be used as an example or used to pull this data. Pulling from this Arkiver: https://data.staging.arkiver.net/robolabs/aperture/graphql

Pulled from an Arkiver which hosts data from Arbitrum including GLP rewards and pricing, ETH pricing data and Lyra trades.

In [1]:
import sys
import os
print(os.getcwd())

import graphql
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
import pandas as pd
import requests
import json

import time

url = 'https://data.staging.arkiver.net/robolabs/aperture/graphql'

D:\Documenten\Programmeren\Python\Cryptocurrency\Arkiver


# GLP and ETH data

In [2]:
# GLP and ETH data

queryStr = '''query MyQuery($limit: Int!, $skip: Int!) {
  HourDatas(limit: $limit, skip: $skip) {
    ethPrice
    glpPrice
    glpTotalSupply
    gmxCumulativeRewards
    gmxEmissions
    gmxEmissionsUsd
    gmxPrice
    timestamp
  }
  }'''


###Grab Data
transport = AIOHTTPTransport(url=url)

client = Client(transport=transport)
query = gql(queryStr)
params = {"limit": 1000, "skip": 0}

# Execute the query on the transport
result = await client.execute_async(query, variable_values=params)

endResult = []
endResult.append(result['HourDatas'])

i = 0

while(len(result['HourDatas']) > 0):
    params["skip"] = params["skip"]+params["limit"]
    
    try: 
        result = await client.execute_async(query, variable_values=params)
        endResult.append(result['HourDatas'])

    except: 
        time.sleep(5)
        result = await client.execute_async(query, variable_values=params)
        endResult.append(result['HourDatas'])

       
    i += 1
    if i % 20 == 0 : 
        print(i)


In [3]:
dfList = []
for itemBatch in endResult:
    df = pd.DataFrame.from_dict(itemBatch)
    dfList.append(df)

df = pd.concat(dfList)
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')

print(df.shape)
print(df['datetime'].max())
print(df['datetime'].min())
df.head()

(3588, 9)
2023-10-28 12:00:00
2022-03-09 16:00:00


,ethPrice,glpPrice,glpTotalSupply,gmxCumulativeRewards,gmxEmissions,gmxEmissionsUsd,gmxPrice,timestamp,datetime
0,2752.9847,1.030440,1.729760e+08,0.000068,0.000000,0.000000,0.097807,1646841600,2022-03-09 16:00:00
1,2708.1653,1.023839,1.733848e+08,0.000068,15.365171,15.365171,0.097807,1646856000,2022-03-09 20:00:00
2,2724.1205,1.025328,1.741920e+08,0.000068,12.732905,12.732905,0.097807,1646870400,2022-03-10 00:00:00
3,2637.4953,1.010068,1.744936e+08,0.000068,14.355209,14.355209,0.097807,1646884800,2022-03-10 04:00:00
4,2580.8332,1.001421,1.751839e+08,0.000068,14.014575,14.014575,0.097807,1646899200,2022-03-10 08:00:00


In [4]:
df.to_csv('HourData.csv', index=False)

# Lyra data

In [5]:
queryStr = '''query MyQuery($limit: Int!, $skip: Int!) {
        Trades(limit: $limit, skip: $skip) {
        block
        timestamp
        trade {
          strikeId
          expiry
          strikePrice
          optionType
          tradeDirection
          amount
          setCollateralTo
          spotPrice
          reservedFee
          totalCost
          isForceClose
        }
        _id
    }
  }'''

###Grab Data

query = gql(queryStr)
params = {"limit": 10000, "skip": 0}

# Execute the query on the transport
result = await client.execute_async(query, variable_values=params)
endResult = []
endResult.append(result['Trades'])

i = 0

while(len(result['Trades']) > 0):
    params["skip"] = params["skip"]+params["limit"]
    
    try: 
        result = await client.execute_async(query, variable_values=params)
        endResult.append(result['Trades'])

    except: 
        time.sleep(5)
        result = await client.execute_async(query, variable_values=params)
        endResult.append(result['Trades'])

       
    i += 1
    if i % 20 == 0 : 
        print(i)


In [6]:
dfList = []
for itemBatch in endResult:
    df = pd.DataFrame.from_dict(itemBatch)
    dfList.append(df)

df = pd.concat(dfList)
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')

print(df.shape)
print(df['datetime'].max())
print(df['datetime'].min())
df.head()

(33054, 5)
2023-10-28 15:05:03
2023-01-24 01:39:33


,block,timestamp,trade,_id,datetime
0,55042396,1674524373,"{'strikeId': 7, 'expiry': 1675688400, 'strikeP...",6515cc5ce9460abbb9c3ae0a,2023-01-24 01:39:33
1,55042419,1674524384,"{'strikeId': 7, 'expiry': 1675688400, 'strikeP...",6515cc5ce9460abbb9c3ae0e,2023-01-24 01:39:44
2,55066773,1674536829,"{'strikeId': 5, 'expiry': 1675688400, 'strikeP...",6515cc68e9460abbb9c3ae76,2023-01-24 05:07:09
3,55290016,1674623090,"{'strikeId': 18, 'expiry': 1674979200, 'strike...",6515ccdae9460abbb9c3b206,2023-01-25 05:04:50
4,55290234,1674623195,"{'strikeId': 16, 'expiry': 1674979200, 'strike...",6515ccdae9460abbb9c3b20b,2023-01-25 05:06:35


In [7]:
df.to_csv('Trades.csv', index=False)